# Weight Optimization Analysis

This notebook analyzes the weight optimization results for the EU Legal Recommender system, focusing on understanding the impact of different weight configurations on recommendation quality.

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Load Optimization Results

In [ ]:
# Load the optimization results
with open('energy_optimization_results.json', 'r') as f:
    results = json.load(f)

# Check which clients are in the results
clients = list(results.keys())
print(f"Clients in results: {clients}")

## Extract and Process Data

Let's extract the key information for each configuration and organize it into DataFrames for analysis.

In [ ]:
def extract_similarity_results(client_results):
    """Extract similarity weight optimization results for precision and NDCG."""
    similarity_results = []
    
    if 'full_profile' in client_results and 'similarity' in client_results['full_profile']:
        similarity = client_results['full_profile']['similarity']
        
        # Process precision results
        if 'precision@10' in similarity:
            precision_data = similarity['precision@10']
            for result in precision_data.get('all_results', []):
                if 'weights' in result and 'score' in result:
                    row = {
                        'text_weight': result['weights'].get('text_weight', 0),
                        'categorical_weight': result['weights'].get('categorical_weight', 0),
                        'precision@10': result.get('score', 0),
                        'metric': 'precision@10'
                    }
                    similarity_results.append(row)
        
        # Process NDCG results
        if 'ndcg@10' in similarity:
            ndcg_data = similarity['ndcg@10']
            for result in ndcg_data.get('all_results', []):
                if 'weights' in result and 'score' in result:
                    row = {
                        'text_weight': result['weights'].get('text_weight', 0),
                        'categorical_weight': result['weights'].get('categorical_weight', 0),
                        'ndcg@10': result.get('score', 0),
                        'metric': 'ndcg@10'
                    }
                    similarity_results.append(row)
    
    return pd.DataFrame(similarity_results) if similarity_results else None

def extract_personalization_results(client_results):
    """Extract personalization weight optimization results for different configurations."""
    personalization_results = []
    
    for config in ['full_profile', 'expert_only', 'categorical_only', 'historical_only']:
        if config in client_results and 'personalization' in client_results[config]:
            personalization = client_results[config]['personalization']
            
            # Process precision results
            if 'precision@10' in personalization:
                precision_data = personalization['precision@10']
                for result in precision_data.get('all_results', []):
                    if 'weights' in result:
                        row = {
                            'expert_weight': result['weights'].get('expert_weight', 0),
                            'historical_weight': result['weights'].get('historical_weight', 0),
                            'categorical_weight': result['weights'].get('categorical_weight', 0),
                            'precision@10': result.get('score', 0),
                            'config': config,
                            'metric': 'precision@10'
                        }
                        personalization_results.append(row)
            
            # Process NDCG results
            if 'ndcg@10' in personalization:
                ndcg_data = personalization['ndcg@10']
                for result in ndcg_data.get('all_results', []):
                    if 'weights' in result:
                        row = {
                            'expert_weight': result['weights'].get('expert_weight', 0),
                            'historical_weight': result['weights'].get('historical_weight', 0),
                            'categorical_weight': result['weights'].get('categorical_weight', 0),
                            'ndcg@10': result.get('score', 0),
                            'config': config,
                            'metric': 'ndcg@10'
                        }
                        personalization_results.append(row)
    
    return pd.DataFrame(personalization_results) if personalization_results else None

def extract_embedding_results(client_results):
    """Extract embedding weight optimization results."""
    embedding_results = []
    
    if 'full_profile' in client_results and 'embedding' in client_results['full_profile']:
        embedding = client_results['full_profile']['embedding']
        
        # Process precision results
        if 'precision@10' in embedding:
            precision_data = embedding['precision@10']
            for result in precision_data.get('all_results', []):
                if 'weights' in result and 'score' in result:
                    row = {
                        'summary_weight': result['weights'].get('summary_weight', 0),
                        'keyword_weight': result['weights'].get('keyword_weight', 0),
                        'precision@10': result.get('score', 0),
                        'metric': 'precision@10'
                    }
                    embedding_results.append(row)
        
        # Process NDCG results
        if 'ndcg@10' in embedding:
            ndcg_data = embedding['ndcg@10']
            for result in ndcg_data.get('all_results', []):
                if 'weights' in result and 'score' in result:
                    row = {
                        'summary_weight': result['weights'].get('summary_weight', 0),
                        'keyword_weight': result['weights'].get('keyword_weight', 0),
                        'ndcg@10': result.get('score', 0),
                        'metric': 'ndcg@10'
                    }
                    embedding_results.append(row)
    
    return pd.DataFrame(embedding_results) if embedding_results else None

# Extract data for the renewable energy client
client = 'renewable_energy_client'
similarity_df = extract_similarity_results(results[client])
personalization_df = extract_personalization_results(results[client])
embedding_df = extract_embedding_results(results[client])

## Analyze Similarity Weight Results

In [ ]:
if similarity_df is not None:
    # Display statistics for similarity weights
    print("Summary statistics for similarity weights:")
    print(similarity_df.describe())
    
    # Find the best weights for precision
    precision_df = similarity_df[similarity_df['metric'] == 'precision@10']
    best_precision = precision_df.sort_values('precision@10', ascending=False).head(5)
    print("\nTop 5 weights for precision@10:")
    print(best_precision)
    
    # Find the best weights for NDCG
    ndcg_df = similarity_df[similarity_df['metric'] == 'ndcg@10']
    if not ndcg_df.empty:
        best_ndcg = ndcg_df.sort_values('ndcg@10', ascending=False).head(5)
        print("\nTop 5 weights for ndcg@10:")
        print(best_ndcg)
    
    # Create a plot of text_weight vs performance
    plt.figure(figsize=(10, 6))
    
    # Plot precision results
    if not precision_df.empty:
        plt.scatter(precision_df['text_weight'], precision_df['precision@10'], 
                   label='Precision@10', alpha=0.7)
    
    # Plot NDCG results if available
    if not ndcg_df.empty:
        plt.scatter(ndcg_df['text_weight'], ndcg_df['ndcg@10'], 
                   label='NDCG@10', alpha=0.7)
    
    plt.xlabel('Text Weight')
    plt.ylabel('Performance Score')
    plt.title('Performance vs. Text Weight')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No similarity weight data available")

## Analyze Personalization Weight Results

In [ ]:
if personalization_df is not None:
    # Number of non-zero results for precision and NDCG
    print("Number of non-zero precision results:")
    precision_counts = personalization_df[personalization_df['metric'] == 'precision@10'].query('`precision@10` > 0').groupby('config').size()
    print(precision_counts)
    
    print("\nNumber of non-zero NDCG results:")
    ndcg_counts = personalization_df[personalization_df['metric'] == 'ndcg@10'].query('`ndcg@10` > 0').groupby('config').size()
    print(ndcg_counts)
    
    # Best configurations for each metric
    print("\nBest configurations for precision@10:")
    best_precision = personalization_df[personalization_df['metric'] == 'precision@10'].sort_values('precision@10', ascending=False).head(5)
    print(best_precision[['config', 'expert_weight', 'historical_weight', 'categorical_weight', 'precision@10']])
    
    print("\nBest configurations for NDCG@10:")
    best_ndcg = personalization_df[personalization_df['metric'] == 'ndcg@10'].sort_values('ndcg@10', ascending=False).head(5)
    print(best_ndcg[['config', 'expert_weight', 'historical_weight', 'categorical_weight', 'ndcg@10']])
    
    # Create a 3D scatter plot for full profile results
    full_profile = personalization_df[personalization_df['config'] == 'full_profile']
    if not full_profile.empty:
        precision_3d = full_profile[full_profile['metric'] == 'precision@10']
        
        if not precision_3d.empty:
            from mpl_toolkits.mplot3d import Axes3D
            
            fig = plt.figure(figsize=(12, 10))
            ax = fig.add_subplot(111, projection='3d')
            
            scatter = ax.scatter(precision_3d['expert_weight'], 
                               precision_3d['historical_weight'], 
                               precision_3d['categorical_weight'],
                               c=precision_3d['precision@10'],
                               cmap='viridis',
                               s=50,
                               alpha=0.7)
            
            ax.set_xlabel('Expert Weight')
            ax.set_ylabel('Historical Weight')
            ax.set_zlabel('Categorical Weight')
            ax.set_title('Personalization Weights vs. Precision@10')
            
            # Add color bar
            cbar = fig.colorbar(scatter, ax=ax, shrink=0.7)
            cbar.set_label('Precision@10')
            
            plt.show()
    
    # Compare performance across configurations
    config_precision = personalization_df[personalization_df['metric'] == 'precision@10'].groupby('config')['precision@10'].max().reset_index()
    config_ndcg = personalization_df[personalization_df['metric'] == 'ndcg@10'].groupby('config')['ndcg@10'].max().reset_index()
    
    plt.figure(figsize=(10, 6))
    
    x = np.arange(len(config_precision))
    width = 0.35
    
    plt.bar(x - width/2, config_precision['precision@10'], width, label='Precision@10')
    plt.bar(x + width/2, config_ndcg['ndcg@10'], width, label='NDCG@10')
    
    plt.xlabel('Configuration')
    plt.ylabel('Performance Score')
    plt.title('Best Performance by Configuration')
    plt.xticks(x, config_precision['config'], rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("No personalization weight data available")

## Analyze Embedding Weight Results

In [ ]:
if embedding_df is not None:
    # Display summary statistics
    print("Summary statistics for embedding weights:")
    print(embedding_df.describe())
    
    # Find the best weights for precision
    precision_df = embedding_df[embedding_df['metric'] == 'precision@10']
    best_precision = precision_df.sort_values('precision@10', ascending=False).head(5)
    print("\nTop 5 weights for precision@10:")
    print(best_precision)
    
    # Find the best weights for NDCG
    ndcg_df = embedding_df[embedding_df['metric'] == 'ndcg@10']
    if not ndcg_df.empty:
        best_ndcg = ndcg_df.sort_values('ndcg@10', ascending=False).head(5)
        print("\nTop 5 weights for ndcg@10:")
        print(best_ndcg)
    
    # Create a plot of summary_weight vs performance
    plt.figure(figsize=(10, 6))
    
    # Plot precision results
    if not precision_df.empty:
        plt.scatter(precision_df['summary_weight'], precision_df['precision@10'], 
                   label='Precision@10', alpha=0.7)
    
    # Plot NDCG results if available
    if not ndcg_df.empty:
        plt.scatter(ndcg_df['summary_weight'], ndcg_df['ndcg@10'], 
                   label='NDCG@10', alpha=0.7)
    
    plt.xlabel('Summary Weight')
    plt.ylabel('Performance Score')
    plt.title('Performance vs. Summary Weight')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No embedding weight data available")

## Analysis of Non-Zero Performing Configurations

In [ ]:
# Analyze all configurations that achieved non-zero performance
if personalization_df is not None:
    # Find all non-zero precision configurations
    non_zero_precision = personalization_df[
        (personalization_df['metric'] == 'precision@10') & 
        (personalization_df['precision@10'] > 0)
    ].copy()
    
    print(f"Found {len(non_zero_precision)} configurations with non-zero precision")
    
    # Find all non-zero NDCG configurations
    non_zero_ndcg = personalization_df[
        (personalization_df['metric'] == 'ndcg@10') & 
        (personalization_df['ndcg@10'] > 0)
    ].copy()
    
    print(f"Found {len(non_zero_ndcg)} configurations with non-zero NDCG")
    
    # Analyze patterns in non-zero configurations
    if not non_zero_precision.empty:
        print("\nSummary of non-zero precision configurations:")
        
        # Group by configuration type
        config_counts = non_zero_precision['config'].value_counts()
        print(f"\nConfiguration counts:\n{config_counts}")
        
        # Calculate weight statistics
        print("\nWeight statistics for non-zero precision configurations:")
        print(non_zero_precision[['expert_weight', 'historical_weight', 'categorical_weight']].describe())
        
        # Create histograms for each weight
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        sns.histplot(non_zero_precision['expert_weight'], bins=10, kde=True, ax=axes[0])
        axes[0].set_title('Expert Weight Distribution')
        
        sns.histplot(non_zero_precision['historical_weight'], bins=10, kde=True, ax=axes[1])
        axes[1].set_title('Historical Weight Distribution')
        
        sns.histplot(non_zero_precision['categorical_weight'], bins=10, kde=True, ax=axes[2])
        axes[2].set_title('Categorical Weight Distribution')
        
        plt.tight_layout()
        plt.show()
    
    # Detailed analysis of NDCG results
    if not non_zero_ndcg.empty:
        print("\nSummary of non-zero NDCG configurations:")
        
        # Group by configuration type
        config_counts = non_zero_ndcg['config'].value_counts()
        print(f"\nConfiguration counts:\n{config_counts}")
        
        # Calculate weight statistics
        print("\nWeight statistics for non-zero NDCG configurations:")
        print(non_zero_ndcg[['expert_weight', 'historical_weight', 'categorical_weight']].describe())
        
        # Plot NDCG values
        plt.figure(figsize=(10, 6))
        sns.boxplot(x='config', y='ndcg@10', data=non_zero_ndcg)
        plt.title('NDCG@10 Distribution by Configuration')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
else:
    print("No personalization data available for non-zero analysis")

## Conclusions

Based on the analysis above, we can draw the following conclusions about the optimal weight configurations for the renewable energy client:

1. **Best Overall Configuration**: [Your conclusion here based on the results]

2. **Expert Profile Importance**: [Your conclusion here based on the results]

3. **Historical vs. Categorical Weights**: [Your conclusion here based on the results]

4. **Text vs. Categorical Similarity**: [Your conclusion here based on the results]

5. **Embedding Weights (Summary vs. Keywords)**: [Your conclusion here based on the results]

6. **Recommendations for Improvement**: [Your recommendations based on the analysis]